In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Install required Dependencies
!pip install mediapipe
!pip install sk-video

In [ ]:
#Import required modules
import os
import cv2 as cv
import numpy as np
import time
import datetime
import math
import random
import time
import datetime
import mediapipe as mp
import numpy as np
from matplotlib import pyplot
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense,Flatten,Dropout,LSTM,TimeDistributed
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import Adam
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
import os

In [ ]:
#object for pose estimation
mp_pose = mp.solutions.pose
pose=mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
currvideo=np.zeros((33*4,))
video_path="/content/drive/Shareddrives/Major project/Major Project/Human Activity Recognition on Yoga Dataset/Yoga dataset(6asanas)/Bhujangasana/Bhujangasana (8).mp4"
cam=cv.VideoCapture(video_path)     #open a video file through the given path
print(video_path)
z=0
while(True):
  ret,frame = cam.read()            #reading a video which returns the current frame and whether next frame is present or not
  if ret:
    if z%2==0:
      arr=np.zeros((33,4))
      results=pose.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
      if results.pose_landmarks is None:
        continue
      image_height, image_width, _ = frame.shape
      j=0
      for i in range(33):
        arr[i,j] = results.pose_landmarks.landmark[i].x*image_width
      j=1
      for i in range(33):
        arr[i,j] = results.pose_landmarks.landmark[i].y*image_height
      j=2
      for i in range(33):
        arr[i,j] = results.pose_landmarks.landmark[i].z*image_width
      j=3
      for i in range(33):
        arr[i][j] = results.pose_landmarks.landmark[i].visibility
      min_x=min(arr[:,0])
      max_x=max(arr[:,0])
      min_y=min(arr[:,1])
      max_y=max(arr[:,1])
      j=0
      for i in range(33):
        arr[i,j]=(256/(max_x-min_x))*(arr[i][j]-min_x)
      j=2
      for i in range(33):
        arr[i,j]=(256/(max_x-min_x))*(arr[i][j]-min_x)
      j=1
      for i in range(33):
        arr[i,j]=(256/(max_y-min_y))*(arr[i][j]-min_y)
      arr=arr.reshape(33*4,)
      currvideo=np.vstack((currvideo,arr))
  else:
    break


/content/drive/Shareddrives/Major project/Major Project/Human Activity Recognition on Yoga Dataset/Yoga dataset(6asanas)/Bhujangasana/Bhujangasana (8).mp4


In [ ]:

"/content/drive/Shareddrives/Major project/Major Project/Human Activity Recognition on Yoga Dataset/Yoga dataset(6asanas)/Bhujangasana/Bhujangasana (8).mp4"
currvideo=currvideo[1:,:]
currvideo.shape
currvideo.shape

(1468, 132)

In [ ]:
windowsize=64           #each windowsize represents a single input
overlap_size=48         #overlap size is how many frames from each adjacent inputs should be common
vid=currvideo
testXfinal=np.zeros((1,64,132))
for start in range(0,vid.shape[0]-windowsize, windowsize - overlap_size):
  currvid=vid[start:start+windowsize,:]
  currvid=currvid.reshape(1,64,132)
  assert currvid.shape==(1,64,132) ,"Something went wrong"
  #print(trainXfinal.shape,currvid.shape)
  testXfinal=np.vstack([testXfinal,currvid])

In [ ]:
print(testXfinal.shape)

(89, 64, 132)


In [ ]:
#defining the model
def get_model(num_len,num_features,num_outputs):
  model = Sequential()
  model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, kernel_initializer='glorot_uniform',activation='relu'), input_shape=(None,num_len,num_features)))
  model.add(TimeDistributed(Conv1D(filters=8, kernel_size=3, activation='relu')))
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(100))
  model.add(Dropout(0.2))
  adam=Adam(lr=0.0001)
  model.add(Dense(num_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
  return model
num_ts, num_features, num_outputs = testXfinal.shape[1], testXfinal.shape[2], 6
num_step, num_len = 4, 16
testXfinal = testXfinal.reshape(testXfinal.shape[0], num_step, num_len, num_features)
model=get_model(num_len,num_features,num_outputs)
model.load_weights("/content/drive/Shareddrives/Major project/Major Project/Human Activity Recognition on Yoga Dataset/weights/val1-83-0.9865.hdf5")
preds = model.predict(testXfinal)

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import skvideo.io
import tqdm
Y_pred_asanas = np.argmax(preds, axis = 1)
answer=np.bincount(Y_pred_asanas).argmax()
asanas = {0:'bhujangasan', 1:'padamasan', 2:'shavasan', 3:'tadasan', 4:'trikonasan', 5:'vrikshasan'}
print("\nName of asana recognized by the model is "+asanas[answer])
def extract_frames(video_path):
    cam=cv.VideoCapture(video_path)
    extract_frames.fps=cam.get(cv.CAP_PROP_FPS)
    #frame_array=[]
    while(True):
      ret,frame = cam.read()
      if ret:
        #frame_array.append(frame)
        yield frame
      else:
        break
font_path = "/content/drive/Shareddrives/Major project/Major Project/RocknRollOne-Regular.ttf"
font = ImageFont.truetype(font_path, 30)
writer = skvideo.io.FFmpegWriter(asanas[answer]+".mp4",outputdict={"-pix_fmt": "yuv420p","-r":str(30)})  #defining the imagewriter
for frame in tqdm.tqdm(extract_frames(video_path), desc="Writing to video"):
  frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
  frame = Image.fromarray(frame)
  d = ImageDraw.Draw(frame)
  d.text(xy=(60,60), text=asanas[answer], fill=(0, 153, 153),font=font)
  writer.writeFrame(np.array(frame))
writer.close()


Name of asana recognized by the model is bhujangasan


Writing to video: 1468it [01:25, 17.27it/s]
